Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Clustering in Spark using TF-IDF and KMeans

## This cell configures the spark session - Do not change


In [21]:
%%configure -f
{
"conf": {
     "spark.rpc.message.maxSize": 1024,
     "spark.kryoserializer.buffer.max": "256m"
   }
}

StatementMeta(, 33, -1, Finished, Available)


## These are the parameters that need to be changed to your values

In [22]:
# The input file name 
input_filename = 'abfss://[].dfs.core.windows.net/kaggle_videos/output/kaggle_videos_all_captions_single.csv/kaggle_videos_all_captions_single.csv'
# The number of clusters - this can be automated or start with a guesstimate
number_of_clusters = 50
# The output directory where the output file will be written to
output_directory = 'abfss://[].dfs.core.windows.net/kaggle_videos/output/'
# The name of the output file
output_filename = 'kaggle_videos_all_captions_text_clustered.csv'

# The blob account url - https://[accountname].blob.core.windows.net
account_url = "https://[].blob.core.windows.net"
# The blob account name = [accountname]
account_name = ''
# The blob account key [iufquq34r423r2==] - used to generate a SAS key
account_key = ''

# The name of the primary ADLS share
file_system_name="share"
# The directory folders where your files reside  
directory_name='bbc'  # kaggle/videos/

# If set to true generate a 3D scatterplot otherwise 2D
SCATTER_PLOT_3D = True
# If this is set to True then the Coalesce notebook will need to be run to merge the partition files into a single file
LOW_MEMORY_MODE = True

# Azure SubscriptionId
subscription_id=""
# AzureML Workspace Resource Group
resource_group=""
# AzureML Workspace Name
workspace_name=""

StatementMeta(DataDiscovery, 33, 1, Finished, Available)

## Track the Experiment in Azure ML

In [ ]:
from azureml.core import Workspace, Experiment, Run
import mlflow

ws = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name)    
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
experiment_name = f"({mssparkutils.runtime.context['notebookname']}_{str(mssparkutils.env.getJobId())})"
mlflow.set_experiment(experiment_name)
mlflow.log_param("input_filename", input_filename)
mlflow.log_param("number_of_clusters", number_of_clusters)
mlflow.log_param("output_directory", output_directory)
mlflow.log_param("output_filename", output_filename)
mlflow.log_param("account_url", account_url)
mlflow.log_param("account_name", account_name)
mlflow.log_param("file_system_name", file_system_name)
mlflow.log_param("directory_name", directory_name)
mlflow.log_param("SCATTER_PLOT_3D", SCATTER_PLOT_3D)
mlflow.log_param("LOW_MEMORY_MODE", LOW_MEMORY_MODE)
params = {
    "sparkpool": mssparkutils.runtime.context['sparkpool'],
    "workspace": mssparkutils.runtime.context['workspace'],
    "notebookname": mssparkutils.runtime.context['notebookname'],
    "isForPipeline": mssparkutils.runtime.context['isForPipeline'],
    "pipelinejobid": mssparkutils.runtime.context['pipelinejobid']
}

mlflow.log_params(params)
mlflow.pyspark.ml.autolog()

## Apply Data Preprocessing, TF-IDF, Kmeans clustering and T-SNE dimensionality reduction

In [23]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, StopWordsRemover, PCA, RegexTokenizer
from pyspark.ml.clustering import LDA, KMeans, BisectingKMeans
from pyspark.ml import Pipeline
from pyspark import SparkContext, SparkConf
import sys
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
import ntpath
import os
from sklearn.manifold import TSNE
import numpy as np

df = spark.read.load(input_filename, header=True, format='csv')

max_features = 2**12

# Apply Data Preprocessing
tokenizer = RegexTokenizer(inputCol="text", outputCol="tokens", gaps=False, minTokenLength=2, toLowercase=True, pattern="[a-zA-Z\-][a-zA-Z\-]{2,}")
remover = StopWordsRemover(inputCol="tokens", outputCol="stopWordsRemovedTokens")
vectorizer = CountVectorizer(inputCol="stopWordsRemovedTokens", outputCol="word_count_vector", minDF=5, maxDF=0.9)

# Apply TF-IFD 
hashingTF = HashingTF(inputCol="stopWordsRemovedTokens", outputCol="rawFeatures", numFeatures=max_features) 
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=10)

# Apply PCA and Kmeans in a pipeline
pca = PCA(k=20, inputCol="features")
pca.setOutputCol("pca_features")

if SCATTER_PLOT_3D:
  pca_2 = PCA(k=3, inputCol="pca_features")
else:  
  pca_2 = PCA(k=2, inputCol="pca_features")
  
pca_2.setOutputCol("pca_scatterplot_features")

kmeans = KMeans(k=number_of_clusters, seed=42, initMode="k-means||", distanceMeasure="euclidean")

pipeline = Pipeline(stages=[tokenizer, remover, vectorizer, hashingTF, idf, pca, kmeans, pca_2])
model = pipeline.fit(df)
df_coords = model.transform(df)


StatementMeta(DataDiscovery, 33, 2, Finished, Available)

## Retrieve Data from Blob Storage

In [24]:

from azure.identity import DefaultAzureCredential
from datetime import datetime, timedelta
from azure.storage.blob import BlobServiceClient, generate_container_sas, BlobSasPermissions
token_credential = DefaultAzureCredential()

blob_service_client = BlobServiceClient(
    account_url=account_url,
    credential=token_credential
)

from azure.storage.filedatalake import DataLakeServiceClient, generate_directory_sas
SAS = generate_directory_sas(
        account_name=account_name,
        file_system_name=file_system_name,
        directory_name=directory_name,
        credential=account_key,
        permission=BlobSasPermissions(read=True),
        expiry=datetime.utcnow() + timedelta(days=100))

SAS_key  = "?" + SAS

storage_path = os.path.join(account_url, file_system_name, directory_name)
SAS_path = []

def build_sas_path(row):
    file_name = ntpath.basename(row)
    return account_url + "/" + file_system_name + "/" + directory_name + "/" + file_name + SAS_key

udf_build_sas_path = udf(build_sas_path, StringType())

def get_X(row):
    return str(row.values[0])

def get_Y(row):
    return str(row.values[1])

def get_Z(row):
    return str(row.values[2])

def remove_separator(row):
    # Clean up the separator from the string
    return row.replace(",", "").replace("\\", "")

udf_get_X = udf(get_X, StringType())
udf_get_Y = udf(get_Y, StringType())
udf_get_Z = udf(get_Z, StringType())

udf_remove_separator = udf(remove_separator, StringType())

if SCATTER_PLOT_3D:
    df_coords = df_coords.withColumn("blob_path", udf_build_sas_path(df_coords.filename)).withColumn("X", udf_get_X(df_coords.pca_scatterplot_features).cast('string')).withColumn("Y", udf_get_Y(df_coords.pca_scatterplot_features).cast('string')).withColumn("Z", udf_get_Z(df_coords.pca_scatterplot_features).cast('string')).withColumn("text", udf_remove_separator(df.text))
else:
    df_coords = df_coords.withColumn("blob_path", udf_build_sas_path(df_coords.filename)).withColumn("X", udf_get_X(df_coords.pca_scatterplot_features).cast('string')).withColumn("Y", udf_get_Y(df_coords.pca_scatterplot_features).cast('string')).withColumn("text", udf_remove_separator(df.text))

df_coords = df_coords.drop('pca_features', 'pca_scatterplot_features', 'tokens', 'word_count_vector', 'pca_features', 'features', 'rawFeatures', 'stopWordsRemovedTokens')


StatementMeta(DataDiscovery, 33, 3, Finished, Available)

## Save the outcome with two operations (usual or coalesce)

In [25]:
if LOW_MEMORY_MODE:
    df_coords.write.mode('overwrite').options(header='true').csv(os.path.join(output_directory, output_filename))
else:
    df1 = df_coords.filter((df_coords.blob_path != 'blob_path'))
    df1.coalesce(1).write.mode('overwrite').options(header='true').csv(os.path.join(output_directory, output_filename))

mlflow.pyspark.ml.mlflow.end_run()

StatementMeta(DataDiscovery, 33, 4, Finished, Available)

# Add Azure Cognitive Search

## Add Search Parameters

In [ ]:
# Azure Search Admin Key
search_admin_key = ""
# The name of the search service
search_service_name = ""
# The Azure Search Query Key
search_query_key = ""

In [ ]:
from synapse.ml.cognitive import *
from pyspark.sql.functions import monotonically_increasing_id, lit

df = df.drop("_c0")

(
    df.withColumn("key", monotonically_increasing_id().cast("string"))
    .withColumn("SearchAction", lit("upload"))
    .writeToAzureSearch(
        subscriptionKey=search_admin_key,
        actionCol="SearchAction",
        serviceName=search_service_name,
        indexName=experiment_name,  # Defaults to the notebook name
        keyCol="key",
    )
)

## Search the generated Azure Search Index

In [ ]:
import requests
from IPython.display import display, Markdown

term_to_search_for = "covid"

url = "https://{}.search.windows.net/indexes/{}/docs/search?api-version=2019-05-06".format(
    search_service_name, experiment_name

)
jdata = requests.post(url, json={"search": term_to_search_for}, headers={"api-key": search_query_key}).json()

for doc in jdata['value']:
    display(Markdown(f'**Search Score {doc["@search.score"]}** Document {doc["filename"]}'))
    display(Markdown(f'{doc["text"]}'))


# Implement Semantic Search

1) [Enable Semantic Search](https://docs.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search) on your search instance

2) [Configure Semantic Search](https://docs.microsoft.com/en-us/azure/search/semantic-how-to-query-request?tabs=semanticConfiguration%2Cportal#create-a-semantic-configuration)

In [ ]:
term_to_search_for = "Whose thumb was fractured?"

url = "https://{}.search.windows.net/indexes/{}/docs/search?api-version=2021-04-30-Preview".format(
    search_service_name, experiment_name
)
jdata = requests.post(url, json={"search": term_to_search_for, "queryType": "semantic", "semanticConfiguration": "config", "queryLanguage": "en-us", "answers": "extractive|count-3",
"captions": "extractive|highlight-true",  "highlightPreTag": "<mark>","highlightPostTag": "</mark>"}, headers={"api-key": search_query_key}).json()

for doc in jdata['value']:
    display(Markdown(f'**Search Score {doc["@search.score"]}** **Search rerankerScore Score {doc["@search.rerankerScore"]}** Document {doc["filename"]}'))
    display(Markdown(f'@search.captions {doc["@search.captions"]}'))
